# GloVeの単語ベクトルによる類似語検索

In [ ]:
from gensim.models import KeyedVectors

glove_vector_file = "es_gensim_glove_vectors.txt"
word = "トラック"
top_k = 30

glove_vectors = KeyedVectors.load_word2vec_format(glove_vector_file, binary=False)
glove_vectors.most_similar(word, [], top_k)

In [ ]:
import numpy as np

def cos_similarity(x, y, eps=1e-8):
    '''コサイン類似度の算出
    :param x: ベクトル
    :param y: ベクトル
    :param eps: ”0割り”防止のための微小値
    :return:
    '''
    nx = x / (np.sqrt(np.sum(x ** 2)) + eps)
    ny = y / (np.sqrt(np.sum(y ** 2)) + eps)
    return np.dot(nx, ny)

def most_similar(query, word_to_id, id_to_word, word_matrix, top=5):
    '''類似単語の検索
    :param query: クエリ（テキスト）
    :param word_to_id: 単語から単語IDへのディクショナリ
    :param id_to_word: 単語IDから単語へのディクショナリ
    :param word_matrix: 単語ベクトルをまとめた行列。各行に対応する単語のベクトル
が格納されていることを想定する
    :param top: 上位何位まで表示するか
    '''
    if query not in word_to_id:
        print('%s is not found' % query)
        return

    print('\n[query] ' + query)
    query_id = word_to_id[query]
    query_vec = word_matrix[query_id]

    vocab_size = len(id_to_word)

    similarity = np.zeros(vocab_size)
    for i in range(vocab_size):
        similarity[i] = cos_similarity(word_matrix[i], query_vec)

    count = 0
    for i in (-1 * similarity).argsort():
        print("i:{}".format(i))
        if id_to_word[i] == query:
            continue
        print(' %s: %s' % (id_to_word[i], similarity[i]))

        count += 1
        if count >= top:
            return

In [ ]:
import pandas as pd
import pickle
import numpy as np

target_data = 1100
top_k = 10

df = pd.read_csv('wakati_category_all.csv')
with open('scdv.pickle', 'rb') as f:
    scdv = pickle.load(f)

sentences_len = len(df)    

sims = np.array([cos_similarity(scdv[target_data], scdv[i]) for i in range(sentences_len)])

topk_index = np.argsort(-sims)[:top_k]

print("{}, {}".format(df['業種(大分類)'][target_data], df['文章'][target_data]))
print("================")
for i in range(top_k):
    index = topk_index[i]
    print("{}, {}, {}".format(sims[index], df['業種(大分類)'][index], df['文章'][index]))
